# Modelo Bigram 

O modelo **Bigram** é uma abordagem probabilística simples usada em Processamento de Linguagem Natural (NLP) para modelar a sequência de palavras em um texto. Ele assume que a probabilidade de uma palavra em uma frase depende apenas da palavra imediatamente anterior. Essa suposição simplifica os cálculos, mas não considera relações mais amplas no contexto.


### Exemplos de uso:
- **Autocompletar textos:** Dado o contexto de uma palavra anterior, o modelo sugere a próxima palavra provável.
- **Correção ortográfica:** Ajuda a prever palavras que fazem mais sentido no contexto.
- **Tradução automática:** Pode ser usado para prever sequências de palavras na língua-alvo.

### Limitações:
- **Falta de contexto:** O modelo só considera a palavra anterior, ignorando palavras mais distantes que podem ser relevantes.
- **Sparsity (esparsidade):** Em grandes vocabulários, muitas combinações de palavras podem nunca ocorrer no conjunto de treinamento, o que dificulta a estimativa das probabilidades.
- **Não captura dependências longas:** Relações entre palavras distantes no texto não são levadas em conta.

### Extensões:
Para superar essas limitações, foram desenvolvidos modelos mais complexos, como **Trigram** (que considera duas palavras anteriores) e modelos baseados em redes neurais, como os **LSTMs** e **Transformers**, que conseguem capturar dependências de longo alcance e fornecer resultados mais robustos.

In [8]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

print(len(text))
print(text[:201])

232326
﻿
  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW


In [3]:
chars = sorted(set(text))
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


Tokenização é o processo de dividir um texto em partes menores, chamadas de tokens. Tokens podem ser palavras, caracteres, ou até subpalavras, dependendo do objetivo.
No caso do código o texto está sendo dividido em caracteres, e cada caractere será convertido para um número (índice) usando uma função encode. Esse processo é essencial para que textos sejam representados numericamente e usados em modelos de aprendizado de máquina.


In [4]:
# criação de um dicionário que mapeia cada caractere para um número
string_to_int = { ch:i for i, ch in enumerate(chars)} # o resultado será algo como: {'a': 0, 'b': 1, ..., 'z': 25}

# criação de um dicionário inverso que mapeia números de volta para os caracteres
int_to_string = { i:ch for i,ch in enumerate(chars)} # exemplo: {0: 'a', 1: 'b', ..., 25: 'z'}

encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join(int_to_string[i] for i in l) # ''.join(...) junta todos os caracteres em uma única string

print(encode('hello'))

[61, 58, 65, 65, 68]


Um tensor é uma estrutura de dados que generaliza vetores (1D) e matrizes (2D) para mais dimensões (N-dimensional arrays). Usada para representar dados, como vetores ou matrizes.

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


Texto -> Tokens -> Tensor 

#### Separação em treino e teste

In [6]:
n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]

O método abaixo é útil em modelos de linguagem para ensinar a prever o próximo caractere ou palavra com base no que veio antes

In [9]:
block_size = 8 # tamanho do bloco que vamos pegar aleatoriamente do texto
batch_size = 4


x = train_data[:block_size]
y = train_data[1:block_size + 1] # pega o texto e desloca uma posição para frente

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"Quando a entrada é {context}, target é {target}")

Quando a entrada é tensor([80]), target é 0
Quando a entrada é tensor([80,  0]), target é 1
Quando a entrada é tensor([80,  0,  1]), target é 1
Quando a entrada é tensor([80,  0,  1,  1]), target é 28
Quando a entrada é tensor([80,  0,  1,  1, 28]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39]), target é 42
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42]), target é 39
Quando a entrada é tensor([80,  0,  1,  1, 28, 39, 42, 39]), target é 44
